<a href="https://colab.research.google.com/github/shukurullo2004/Machine-learnings/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
print("Training dataset size:", len(train_dataset))
print("Testing dataset size:", len(test_dataset))

for batch in train_loader:
    inputs, labels = batch
    print("Training input shape:", inputs.shape)
    print("Training labels shape:", labels.shape)
    break

for batch in test_loader:
    inputs, labels = batch
    print("Testing input shape:", inputs.shape)
    print("Testing labels shape:", labels.shape)
    break

In [ ]:
from torch import nn
import torch.nn.init as init

class ModelV2(nn.Module):
    def __init__(self):
        super(ModelV2, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)


        init.xavier_uniform_(self.fc1.weight)
        init.xavier_uniform_(self.fc2.weight)
        init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

mymodel = ModelV2()
mymodel

In [ ]:
import torch

def accuracy_fn(y_true, y_pred):
    predicted_classes = torch.argmax(y_pred, dim=1)
    correct = torch.eq(y_true, predicted_classes).sum().item()
    acc = (correct / len(y_true)) * 100
    return acc

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mymodel.parameters(), lr = 0.0001)


In [ ]:
for X_train, y_train in train_loader:
  X_train[:1],y_train[:1]

In [ ]:
epoch_count = []
loss_values = []
test_loss_values = []
epochs = 9
for epoch in range(epochs):
  mymodel.train()
  for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = mymodel(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

  # y_pred = mymodel(X_train).squeeze()

  # acc = accuracy_fn(y_train, y_pred)
  # loss = loss_fn(y_pred, y_train)
  # optimizer.zero_grad()
  # loss.backward()
  # optimizer.step()


  mymodel.eval()
  with torch.inference_mode():
    for X_test, y_test in test_loader:
      y_test_pred = mymodel(X_test)
      test_loss = loss_fn(y_test_pred, y_test)
      test_acc = accuracy_fn(y_test, y_test_pred)

  if epoch % 1 == 0:
            epoch_count.append(epoch)
            loss_values.append(loss)
            test_loss_values.append(test_loss)
            print(f"Epoch: {epoch} | Train Loss: {loss} | Test Loss: {test_loss} | Test acc: {test_acc}| Train acc: {acc}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.plot(epoch_count,np.array(torch.tensor(loss_values)),label = "Train loss")
plt.plot(epoch_count,test_loss_values,label = "Test loss")

In [ ]:
y_train[:5], y_test[:5]

In [ ]:
y_pred[:1]

In [ ]:
outputs = mymodel(inputs)

_, predicted = torch.max(outputs, 1)


predicted = predicted.view(-1)


correct_predictions = torch.eq(predicted, labels)

print("Correct Predictions:", correct_predictions)

In [ ]:
true_labels = []
predicted_labels = []
images = []

outputs = mymodel(inputs)


_, predicted = torch.max(outputs, 1)


true_labels.extend(labels.cpu().numpy())
predicted_labels.extend(predicted.cpu().numpy())


images.extend(inputs[:5])


true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)


plt.figure(figsize=(12, 4))


plt.subplot(1, 2, 1)
plt.scatter(range(len(true_labels)), true_labels, label='True Labels', color='blue', alpha=0.5)
plt.title('True Labels')
plt.xlabel('Sample Index')
plt.ylabel('Label')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(range(len(predicted_labels)), predicted_labels, label='Predicted Labels', color='red', alpha=0.5)
plt.title('Predicted Labels')
plt.xlabel('Sample Index')
plt.ylabel('Label')
plt.legend()


plt.tight_layout()
plt.show()


In [ ]:
plt.subplot(1, 3, 3)
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[i][0], cmap='gray')
    plt.title(f'True: {true_labels[i]}, Pred: {predicted_labels[i]}')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
from pathlib import Path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents =True, exist_ok = True)

MODEL_NAME = "Shukurullos 1 saved model.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

torch.save(obj = mymodel.state_dict,f =MODEL_SAVE_PATH)